In [1]:
import os
import random
from glob import glob
from pathlib import Path
import yaml
from tqdm import tqdm
import IPython.display as ipd
import json

import numpy as np
import pandas as pd

import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt

import librosa
import wandb
from sklearn import metrics

import torch
from torch import nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler

from timm.scheduler import CosineLRScheduler

In [2]:
from config import CFG
from pytorch_model import BirdCLEF23Net
from pytorch_wav2logmel import Wav2Logmel
import pytorch_modeler as modeler
import pytorch_preprocessing as prep
import common as com

In [3]:
modeler.set_seed(CFG.seed)
# setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print('Debug :', CFG.debug)

In [4]:
# Try to get the API key from Kaggle secrets
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    # Login to wandb with the API key
    wandb.login(key=api_key)
    print('kaggle notebook mode')
except:
    key_path = '/kaggle/input/wandb_key.txt'
    p = Path(key_path)
    api_key = p.read_text()
    wandb.login(key=api_key)
    print('local mode')

In [5]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを設定
jst = pytz.timezone('Asia/Tokyo')

# 現在時刻を取得し、日本時間に変換
now = datetime.now(jst)

# 現在時刻を文字列に変換
now_str = now.strftime('%Y-%m-%d %H:%M:%S')

print(now_str)

In [6]:
import yaml
from tqdm import tqdm

def wandb_init(fold):
    config = {k: v for k, v in dict(vars(CFG)).items() if '__' not in k}
    config.update({"fold": int(fold)})
    yaml.dump(config, open(f'./config fold-{fold}.yaml', 'w'), )
    config = yaml.load(open(f'./config fold-{fold}.yaml', 'r'), Loader=yaml.FullLoader)
    run = wandb.init(project="birdclef-2023-public",
                     name=f"fold-{fold}|dim-{CFG.img_size[1]}x{CFG.img_size[0]}|model-{CFG.model_name}|{now_str}",
                     config=config,
                     group=CFG.comment,
                     save_code=True, )
    return run


def log_wandb(valid_df):
    save_df = valid_df.query("miss==True")
    save_df.loc[:, 'pred_name'] = save_df.pred.map(CFG.label2name2)
    save_df.loc[:, 'target_name'] = save_df.target.map(CFG.label2name2)
    if CFG.debug:
        save_df = save_df.iloc[:CFG.batch_size * CFG.valid_bs]
    noimg_cols = [*CFG.tab_cols, 'target', 'pred', 'target_name', 'pred_name']
    save_df = save_df.loc[:, noimg_cols]

#    data = []
#   for idx, row in tqdm(save_df.iterrows(), total=len(save_df), desc='wandb ', position=0, leave=True):
#        filepath = row.
#       audio, sr = librosa.load(filepath, sr=None)
#        data += [[*row.tolist(), wandb.Audio(audio, caption=row.filename, sample_rate=sr)]]
    #wandb_table = wandb.Table(data=data, columns=[*noimg_cols, 'audio'])
    wandb.log({'best': scores,
 #              'table': wandb_table,
               })

In [7]:
# df_23 = pd.read_csv(f'{CFG.BASE_PATH}/train_metadata.csv')
# filename = df_23.filename.str.replace('.ogg', '.wav')
# df_23['filepath'] = CFG.BASE_PATH + '/train_audio_wav/' + filename
# df_23['target'] = df_23.primary_label.map(CFG.name2label)
# df_23['birdclef'] = '23'
# df_23['filename'] = df_23.filepath.map(lambda x: x.split('/')[-1])
# df_23['xc_id'] = df_23.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
# df_23.head(2)

# # Display rwos
# print("# Samples in BirdCLEF 23: {:,}".format(len(df_23)))
# df_23.head(2).style.set_caption("BirdCLEF - 23").set_table_styles([{
#     'selector': 'caption',
#     'props': [
#         ('color', 'blue'),
#         ('font-size', '16px')
#     ]
# }])
df_23 = pd.read_csv(f'{CFG.BASE_PATH3}/train_metadata.csv')
df_23['filepath'] = CFG.BASE_PATH3 + '/train_audio/' + df_23.filename
df_23['target'] = df_23.primary_label.map(CFG.name2label)
df_23['birdclef'] = '23'
df_23['filename'] = df_23.filepath.map(lambda x: x.split('/')[-1])
df_23['xc_id'] = df_23.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
#assert tf.io.gfile.exists(df_23.filepath.iloc[0])

# Display rwos
print("# Samples in BirdCLEF 23: {:,}".format(len(df_23)))
df_23.head(2).style.set_caption("BirdCLEF - 23").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '16px')
    ]
}])

In [8]:
# BirdCLEF-2020
df_20 = pd.read_csv(f'{CFG.BASE_PATH0}/train.csv')
df_20['primary_label'] = df_20['ebird_code']
df_20['filepath'] = CFG.BASE_PATH0 + '/train_audio/' + df_20.primary_label + '/' + df_20.filename
df_20['scientific_name'] = df_20['sci_name']
df_20['common_name'] = df_20['species']
df_20['target'] = df_20.primary_label.map(CFG.name2label2)
df_20['birdclef'] = '20'
#assert tf.io.gfile.exists(df_20.filepath.iloc[0])

# Xeno-Canto Extend by @vopani
df_xam = pd.read_csv(f'{CFG.BASE_PATH4}/train_extended.csv')
df_xam['filepath'] = CFG.BASE_PATH4 + '/A-M/' + df_xam.ebird_code + '/' + df_xam.filename
df_xnz = pd.read_csv(f'{CFG.BASE_PATH5}/train_extended.csv')
df_xnz['filepath'] = CFG.BASE_PATH5 + '/N-Z/' + df_xnz.ebird_code + '/' + df_xnz.filename
df_xc = pd.concat([df_xam, df_xnz], axis=0, ignore_index=True)
df_xc['primary_label'] = df_xc['ebird_code']
df_xc['scientific_name'] = df_xc['sci_name']
df_xc['common_name'] = df_xc['species']
df_xc['target'] = df_xc.primary_label.map(CFG.name2label2)
df_xc['birdclef'] = 'xc'
#assert tf.io.gfile.exists(df_xc.filepath.iloc[0])

# BirdCLEF-2021
df_21 = pd.read_csv(f'{CFG.BASE_PATH1}/train_metadata.csv')
df_21['filepath'] = CFG.BASE_PATH1 + '/train_short_audio/' + df_21.primary_label + '/' + df_21.filename
df_21['target'] = df_21.primary_label.map(CFG.name2label2)
df_21['birdclef'] = '21'
corrupt_paths = [f'{CFG.BASE_PATH1}/train_short_audio/houwre/XC590621.ogg',
                 f'{CFG.BASE_PATH1}/train_short_audio/cogdov/XC579430.ogg']
df_21 = df_21[~df_21.filepath.isin(corrupt_paths)] # remove all zero audios
#assert tf.io.gfile.exists(df_21.filepath.iloc[0])

# BirdCLEF-2022
df_22 = pd.read_csv(f'{CFG.BASE_PATH2}/train_metadata.csv')
df_22['filepath'] = CFG.BASE_PATH2 + '/train_audio/' + df_22.filename
df_22['target'] = df_22.primary_label.map(CFG.name2label2)
df_22['birdclef'] = '22'
#assert tf.io.gfile.exists(df_22.filepath.iloc[0])

# Merge 2021 and 2022 for pretraining
df_pre = pd.concat([df_20, df_21, df_22, df_xc], axis=0, ignore_index=True)
df_pre['filename'] = df_pre.filepath.map(lambda x: x.split('/')[-1])
df_pre['xc_id'] = df_pre.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
nodup_idx = df_pre[['xc_id','primary_label','author']].drop_duplicates().index
df_pre = df_pre.loc[nodup_idx].reset_index(drop=True)

# # Remove duplicates
df_pre = df_pre[~df_pre.xc_id.isin(df_23.xc_id)].reset_index(drop=True)
corrupt_mp3s = json.load(open('/kaggle/input/birdclef-2023-dataset/corrupt_mp3_files.json','r'))
corrupt_mp3s = [path.replace('/kaggle/input','/kaggle/input/birdclef-2023-dataset') for path in corrupt_mp3s]
df_pre = df_pre[~df_pre.filepath.isin(corrupt_mp3s)]
df_pre = df_pre[['filename','filepath','primary_label','secondary_labels',
                 'rating','author','file_type','xc_id','scientific_name',
                'common_name','target','birdclef','bird_seen']]
# Display rows
print("# Samples for Pre-Training: {:,}".format(len(df_pre)))
df_pre.head(2).style.set_caption("Pre-Training Data").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '16px')
    ]
}])

# Show distribution
plt.figure(figsize=(8, 4))
df_pre.birdclef.value_counts().plot.bar(color=[cmap(0.0),cmap(0.25), cmap(0.65), cmap(0.9)])
plt.xlabel("Dataset")
plt.ylabel("Count")
plt.title("Dataset distribution for Pre-Training")
plt.show()

# サンプル数が合わない

In [9]:
df_pre['filepath']=df_pre['filepath'].str.replace('train_audio', 'train_audio_mono_wav')
df_pre['filepath']=df_pre['filepath'].str.replace('train_short_audio', 'train_short_audio_mono_wav')
df_pre['filepath']=df_pre['filepath'].str.replace('A-M', 'A-M_wav')
df_pre['filepath']=df_pre['filepath'].str.replace('mp3', 'wav')
df_pre['filepath']=df_pre['filepath'].str.replace('ogg', 'wav')
df_pre=df_pre.fillna('')

In [10]:
#df_pre.filepath.to_list()

In [11]:
# # Import required packages
# from sklearn.model_selection import StratifiedKFold

# # Initialize the StratifiedKFold object with 5 splits and shuffle the data
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=CFG.seed)

# # Reset the index of the dataframe
# df = df.reset_index(drop=True)

# # Create a new column in the dataframe to store the fold number for each row
# df["fold"] = -1

# # Iterate over the folds and assign the corresponding fold number to each row in the dataframe
# for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
#     df.loc[val_idx, 'fold'] = fold

# Import required packages
from sklearn.model_selection import StratifiedKFold

# Initialize the StratifiedKFold object with 5 splits and shuffle the data
skf1 = StratifiedKFold(n_splits=25, shuffle=True, random_state=CFG.seed)
skf2 = StratifiedKFold(n_splits=CFG.num_fold, shuffle=True, random_state=CFG.seed)

# Reset the index of the dataframe
df_pre = df_pre.reset_index(drop=True)
df_23 = df_23.reset_index(drop=True)

# Create a new column in the dataframe to store the fold number for each row
df_pre["fold"] = -1
df_23["fold"] = -1

# BirdCLEF - 21 & 22
for fold, (train_idx, val_idx) in enumerate(skf1.split(df_pre, df_pre['primary_label'])):
    df_pre.loc[val_idx, 'fold'] = fold
    
# IBirdCLEF - 23
for fold, (train_idx, val_idx) in enumerate(skf2.split(df_23, df_23['primary_label'])):
    df_23.loc[val_idx, 'fold'] = fold

In [12]:
def plot_batch(batch, row=3, col=3, label2name=None,):
    """Plot one batch data"""
    if isinstance(batch, tuple) or isinstance(batch, list):
        audios, tars = batch
    else:
        audios = batch
        tars = None
    plt.figure(figsize=(col*5, row*3))
    for idx in range(row*col):
        ax = plt.subplot(row, col, idx+1)
        plt.plot(audios[idx].numpy(), color=cmap(0.1))
        if tars is not None:
            label = tars[idx].numpy().argmax()
            name = label2name[label]
            plt.title(name)
    plt.tight_layout()
    plt.show()
    
    
def plot_history(history):
    """Plot trainign history, credit: @cdeotte"""
    epochs = len(history.history['auc'])
    plt.figure(figsize=(15,5))
    plt.plot(np.arange(epochs),history.history['auc'],'-o',label='Train AUC',color='#ff7f0e')
    plt.plot(np.arange(epochs),history.history['val_auc'],'-o',label='Val AUC',color='#1f77b4')
    x = np.argmax( history.history['val_auc'] ); y = np.max( history.history['val_auc'] )
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
    plt.ylabel('AUC (PR)',size=14); plt.xlabel('Epoch',size=14)
    plt.legend(loc=2)
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(epochs),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
    plt2.plot(np.arange(epochs),history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
    x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    plt.ylabel('Loss',size=14)
    plt.title('Fold %i - Training Plot'%(fold+1),size=18)
    plt.legend(loc=3)
    plt.show()  

In [13]:
import gc
def gc_collect():
    gc.collect()
    torch.cuda.empty_cache()

In [14]:
#%env CUDA_LAUNCH_BLOCKING=1

oof_pred = []; oof_true = []; oof_val = []; oof_ids = []; oof_folds = [] 
# Configurations
num_classes = CFG.num_classes2
df = df_pre.copy()
fold = 0

scaler = torch.cuda.amp.GradScaler()

# Initialize Weights and Biases
if CFG.wandb:
    run = wandb_init(fold)

# Compute batch size and number of samples to drop
infer_bs = CFG.valid_bs
drop_remainder = CFG.drop_remainder

# Split dataset with cv filter
if CFG.cv_filter:
    df = com.filter_data(df, thr=5)
    train_df = df.query("fold!=@fold | ~cv").reset_index(drop=True)
    valid_df = df.query("fold==@fold & cv").reset_index(drop=True)
else:
    train_df = df.query("fold!=@fold").reset_index(drop=True)
    valid_df = df.query("fold==@fold").reset_index(drop=True)

# Upsample train data
train_df = com.upsample_data(train_df, thr=50)
train_df = com.downsample_data(train_df, thr=500)

# Get file paths and labels
train_paths = train_df.filepath.values; train_labels = train_df.target.values
valid_paths = valid_df.filepath.values; valid_labels = valid_df.target.values

# Shuffle the file paths and labels
index = np.arange(len(train_paths))
np.random.shuffle(index)
train_paths  = train_paths[index]
train_labels = train_labels[index]

# wav
train_ftype = list(map(lambda x: '.wav' in x, train_paths))
valid_ftype = list(map(lambda x: '.wav' in x, valid_paths))

# Compute the number of training and validation samples
num_train = len(train_paths); num_valid = len(valid_paths)
    
# Log the number of training and validation samples if Weights and Biases is being used
if CFG.wandb:
    wandb.log({'num_train':num_train,
                'num_valid':num_valid})
    
# Build the training and validation datasets
# For debugging
if CFG.debug:
    min_samples = CFG.batch_size
    train_ds = prep.BirdDataset(train_df.iloc[:min_samples], is_train=True)
    valid_ds = prep.BirdDataset(valid_df, is_train=False)
else:
    train_ds = prep.BirdDataset(train_df, is_train=True)
    valid_ds = prep.BirdDataset(valid_df, is_train=False)
# dataloader
train_dataloader, val_dataloader = modeler.make_dataloder(train_ds, valid_ds)

wav_to_logmel = Wav2Logmel()
# Clear the session and build the model
model = BirdCLEF23Net(num_classes=num_classes)
# Load birdclef pretrained weights
if CFG.pretrain == True:
    model.load_state_dict(torch.load(CFG.pretrained_model_path), strict=False)
model.to(device)
wav_to_logmel.to(device)

print('#' * 25)
print('#### Training')
print('#### Fold: %i | Image Size: (%i, %i) | Model: %s | Batch Size: %i | Scheduler: %s' %
    (fold + 1, *CFG.img_size, CFG.model_name, CFG.batch_size, CFG.scheduler))
print('#### Num Train: {:,} | Num Valid: {:,}'.format(len(train_paths), len(valid_paths)))

optimizer = com.get_optimizer(model)
# TODO com.get_scheduler
scheduler = CosineLRScheduler(optimizer, t_initial=CFG.epochs, lr_min=CFG.lr_min, 
                                warmup_t=CFG.warmup_t, warmup_lr_init=CFG.warmup_lr_init, warmup_prefix=True)
criterion = com.get_criterion()

best_score = -1
best_epoch = -1

for epoch in range(2 if CFG.debug else CFG.epochs):
    # Training
    model.train()
    epoch_loss = 0
    for batch_idx, (inputs, sample_info) in enumerate(tqdm(train_dataloader)):
        inputs, targets = inputs.to(device), sample_info['target'].to(device)
        targets = F.one_hot(targets, num_classes=num_classes).float()
        optimizer.zero_grad()
        logmel = wav_to_logmel(inputs)
        with torch.cuda.amp.autocast():
            outputs, mix_targets = model(logmel, targets)
            loss = criterion(outputs, mix_targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if np.isinf(loss.item()) or np.isnan(loss.item()):
            print(f'Bad loss, skipping the batch {batch_idx}')
            del loss, outputs, mix_targets
            gc_collect()
            continue
        epoch_loss += loss.item()
        # wandb logger (Train loss)
        run.log({'loss': loss.item()})
    scheduler.step(epoch+1)
    
    # gc
    # del logmel
    # torch.cuda.empty_cache()
    # Validation
    model.eval()
    val_loss = 0
    val_preds = []
    val_true = []
    with torch.no_grad():
        for inputs, sample_info in tqdm(val_dataloader):
            inputs, targets = inputs.to(device), sample_info['target'].to(device)
            targets = F.one_hot(targets, num_classes=num_classes).float()
            logmel = wav_to_logmel(inputs)
            outputs = model(logmel)
            loss = criterion(outputs, targets)
            outputs = torch.softmax(outputs, dim=1)#torch.softmax(outputs, dim=1)
            #outputs = torch.softmax(outputs, dim=1)
            val_loss += loss.item()
            val_preds.append(outputs.detach().cpu().numpy())
            val_true.append(targets.detach().cpu().numpy())

    val_preds = np.vstack(val_preds)
    val_true = np.vstack(val_true)
    # Metrics
    val_score = com.padded_cmap(val_true, val_preds)
    # Checkpoint
    if val_score > best_score:
        best_score = val_score
        best_epoch = epoch + 1
        torch.save(model.state_dict(), f'fold-{fold}.pth')
        art = wandb.Artifact("birdclef-2023", type="model")
        art.add_file(f'fold-{fold}.pth')
        run.log_artifact(art)

    print(f'Epoch: {epoch + 1} | Train Loss: {epoch_loss / len(train_dataloader)} | '
        f'Val Loss: {val_loss / len(val_dataloader)} | Val Padded_cmAP : {val_score}')
    
    # wandb logger
    lr = scheduler.get_epoch_values(epoch)[0]
    run.log({'train_loss': epoch_loss / len(train_dataloader),
             'lr': lr,
             'epoch': epoch+1,
             'valid_loss': val_loss / len(val_dataloader),
             'valid_padded_cmAP': val_score,})
    
    
# Load best checkpoint
print('# Loading best model')
model.load_state_dict(torch.load(f'fold-{fold}.pth'))

# Predict on the validation data for oof result
print('# Infering OOF')
model.eval()
oof_pred_ = []
with torch.no_grad():
    for inputs, sample_info in tqdm(val_dataloader):
        inputs, targets = inputs.to(device), sample_info['target'].to(device)
        logmel = wav_to_logmel(inputs)
        outputs = model(logmel)
        outputs = torch.softmax(outputs, dim=1)
        oof_pred_.append(outputs.detach().cpu().numpy())

oof_pred_ = np.concatenate(oof_pred_, axis=0)
# oof_pred : 5
oof_pred.append(oof_pred_)

# Get ids and targets
oof_true.append(valid_labels)
oof_folds.append(np.ones_like(oof_true[-1], dtype='int8') * fold)
oof_ids.append(valid_paths)

# Save valid data prediction
y_true = np.array(oof_true[-1])
y_pred = np.argmax(oof_pred[-1], axis=-1)

valid_df['pred'] = y_pred
valid_df['miss'] = y_true != y_pred
valid_df[CFG.class_names2] = oof_pred[-1].tolist()
# Log the metrics
scores = {}
cmAP = com.padded_cmap(com.one_hot_encode(y_true), oof_pred[-1])
oof_val.append(best_score)
print('\n>>> FOLD %i Padded_cmAP = %.3f' % (fold+1, cmAP))
scores.update({'epoch': best_epoch,
                'cmAP': cmAP,})
# wandb logger 
run.log(scores)
# Show training plot
# if CFG.training_plot:
#     plot_history(history)
# Log metrics, media to wandb
if CFG.wandb:
    print('# WandB')
    log_wandb(valid_df)
    wandb.run.finish()
    #display(ipd.IFrame(run.url, width=1080, height=720))